In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
#%matplotlib inline
import cv2

# For one hot encoding
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [2]:
import pickle

def save_to_pickle(path = None, name = None, data = None):
    with open(path+name, "wb") as f:
        print("\t => Save '{0}' to '{1}'".format(name, path))
        pickle.dump(data, f)
        
def unpickle(file):
    with open(file, 'rb') as fo:
        dictionary = pickle.load(fo, encoding='bytes')
    return dictionary

In [3]:
import os

#os.listdir("/mnt/SharedData/Development/Personal_Dev/Machine-Learning/Data/Face/Face-SJC/")
#os.listdir("/Users/user170/Developments/Personal-Dev./Machine-Learning/Data/Face-SJC")

root_dir = "/Users/user170/Developments/Personal-Dev./Machine-Learning/Data/Face-SJC/"

### dataset preprocessing: merge 100 sample images (converting colour map, resize)

In [4]:
def face_sampling(root_path, sample_size):
    ref_idx = 0
    ref_dict = {}
    labels = []
    imgs_cv = []
    
    root_path = os.listdir(root_dir)
    
    for child in tqdm(root_path):
        child_path = root_dir + child
        if os.path.isdir(child_path):
            img_names = np.array(os.listdir(child_path))
            
            # Sample
            batch_index = np.random.choice(len(img_names), size=sample_size)
            img_names = img_names[batch_index]
            
            for img_name in img_names:
                img_cv = cv2.imread(child_path+'/'+img_name, -1)
                
                # Convert rgba -> bgr
                img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGBA2BGR)
                
                # Reshape image to 224x224
                img_cv = cv2.resize(img_cv,(224, 224), interpolation = cv2.INTER_CUBIC)
                
                imgs_cv.append(img_cv)
                labels.append(ref_idx) # save a label

            #imgs_cv = np.array(imgs_cv)
            #print(imgs_cv.shape)
            #save_to_pickle(root_dir, child+".pkl", imgs_cv)
            
            #print(child_path)
            # Make label reference dictionary
            ref_dict[ref_idx] = child+".pkl"
            ref_idx += 1

    # Convert list to numpy array
    imgs_cv = np.array(imgs_cv)
    labels = np.array(labels)
    
    # Shuffle the dataset
    shuffle_idx = np.arange(len(imgs_cv))
    np.random.shuffle(shuffle_idx)
    imgs_cv = imgs_cv[shuffle_idx]
    labels = labels[shuffle_idx]
    
    # Save to pickle
    save_to_pickle(root_dir, "employee-faces.pkl", {"data":imgs_cv, "labels":labels, "ref_labels":ref_dict})
    
    return imgs_cv, labels, ref_dict
            
features, labels, references = face_sampling(root_dir, 100)


100%|██████████| 11/11 [00:08<00:00,  1.29it/s]


	 => Save 'employee-faces.pkl' to '/Users/user170/Developments/Personal-Dev./Machine-Learning/Data/Face-SJC/'


In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(20, 10))

for i in range(10):
    idx = np.random.randint(len(labels))
    ax[i].imshow(features[idx])
    ax[i].set_title(references[labels[idx]])